In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, glob
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls
from plotly.exceptions import PlotlyEmptyDataError
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from parsing.argus_analysis import *
import numpy as np
from plotting.plt_utils import get_random_colors

tls.set_credentials_file(username='cguldner', api_key=os.environ['PLOTLY_API_KEY'])
init_notebook_mode(connected=True)         # initiate notebook for offline plot

In [5]:
daemon_path = 'data/argusout/'
module_path = 'data/moduleout/'
# How many times per second the daemon collects data, as specified in argus.conf
daemon_sample_rate = 2.0

In [ ]:
basic_plot_params = {
   'axes.labelsize': 14,
   'font.size': 15,
   'legend.fontsize': 11,
   'legend.handlelength': 2,
   'xtick.labelsize': 12,
   'ytick.labelsize': 12,
   'lines.marker': None,
   'text.usetex': False,
   'figure.figsize': [20, 20]
}

plt.rcParams.update(basic_plot_params)

In [ ]:
# Won't work unless we can downgrade matplotlib to version 1.3.1. Use the native plotly plots instead
def plot_daemon_stats(stats, node_names):
    cpu = plt.subplot2grid((3,3), (0,0), colspan=3)
    cpu.set_ylabel('CPU usage')
    cpu.set_title('Percentage of CPU usage')
    
    mem = plt.subplot2grid((3,3), (1,0), colspan=3)
    mem.set_ylabel('Memory (in bytes)')
    mem.set_title('Active Memory usage')

    failed = plt.subplot2grid((3,3), (2, 0))
    failed.set_ylabel('Puzzles')
    failed.set_title('Puzzles Failed')

    received = plt.subplot2grid((3,3), (2, 1))
    received.set_title('Puzzles Received')

    sent = plt.subplot2grid((3,3), (2, 2))
    sent.set_title('Puzzles Sent')
    
    for node in node_names:
        cur_node = stats.get(node)
        try:
            cpu.plot(cur_node['Timestamp'], cur_node['cpu_percent'], label=node+'-cpu')
            mem.plot(cur_node['Timestamp'], cur_node['active_memory'], label=node+'-mem')
            failed.plot(cur_node['Timestamp'], cur_node['ChallengeFailed'], label=node+'-failed')
            received.plot(cur_node['Timestamp'], cur_node['ChallengeRecvd'], label=node+'-received')
            sent.plot(cur_node['Timestamp'], cur_node['ChallengeSent'], label=node+'-sent')
        except KeyError:
            continue
    
    # Print the averages of all the client and attack nodes for cpu
    for node_type in ['client', 'attack']:
        cpu_vals = [np.array(node['cpu_percent']) for name, node in stats.items() if node_type in name]
        longest = max([len(a) for a in cpu_vals])
        cpu_resized = [np.resize(a, longest) for a in cpu_vals]
        cpu.plot(range(longest), np.mean(cpu_resized, axis=0), label='avg-%s-cpu' % node_type)
    
    for plot in [cpu, mem, failed, received, sent]:
        plot.set_xlabel('Time')
        plot.yaxis.grid()
        
    fig = plt.gcf()
    fig.tight_layout(pad=1, h_pad=3.5)
    
    plotly_fig = tls.mpl_to_plotly(fig)
    plotly_fig.layout.showlegend=True
    plotly_fig.layout.autosize=True
    del plotly_fig.layout['width']
    
    up_plt_fig = dict(data=plotly_fig.data, layout=plotly_fig.layout)
    iplot(up_plt_fig, filename='daemon_data', link_text='Export')

In [ ]:
daemon_stats = {}
files = [os.path.splitext(f) for f in os.listdir(daemon_path) if os.path.isfile(daemon_path + f)]
for f in files:
    daemon_stats[f[0]] = get_daemon_stats_from_file(daemon_path + ''.join(f))
    
try:
    plot_daemon_stats(daemon_stats, [f[0] for f in files])
except PlotlyEmptyDataError:
    print('There was no daemon data at the specified location')

In [8]:
def plot_daemon_stats_plotly(stats, node_names):
    fig = tls.make_subplots(rows=3, cols=3, print_grid=False, specs=[
        [{'colspan': 3}, None, None], [{'colspan': 3}, None, None], [{}, {}, {}] ],
        subplot_titles=('Percentage of CPU usage','Active Memory Usage', 'Puzzles Failed', 'Puzzles Received', 'Puzzles Sent')
    )
    for node in node_names:
        cur_node = stats.get(node)
        time_axis = np.asarray(cur_node['Timestamp']) / daemon_sample_rate
        try:
            cpu = go.Scatter(x=time_axis, y=cur_node['cpu_percent'], name=node+'-cpu', line=dict(width=1), xaxis='x1', yaxis='y1')
            fig.append_trace(cpu, 1, 1)
            
            mem = go.Scatter(x=time_axis, y=cur_node['active_memory'], name=node+'-mem', line=dict(width=1), xaxis='x2', yaxis='y2')
            fig.append_trace(mem, 2, 1)
            
            failed = go.Scatter(x=time_axis, y=cur_node['ChallengeFailed'], name=node+'-failed', line=dict(width=1), xaxis='x3', yaxis='y3')
            fig.append_trace(failed, 3, 1)
            
            received = go.Scatter(x=time_axis, y=cur_node['ChallengeRecvd'], name=node+'-received', line=dict(width=1), xaxis='x4', yaxis='y4')
            fig.append_trace(received, 3, 2)
            
            sent = go.Scatter(x=time_axis, y=cur_node['ChallengeSent'], name=node+'-sent', line=dict(width=1), xaxis='x5', yaxis='y5')
            fig.append_trace(sent, 3, 3)
        except KeyError:
            continue

    # Print the averages of all the client and attack nodes for cpu
    for node_type in ['client', 'attack']:
        cpu_vals = np.asarray([node['cpu_percent'] for name, node in stats.items() if node_type in name])
        longest = max([len(a) for a in cpu_vals])
        cpu_resized = [np.resize(a, longest) for a in cpu_vals]
        cpu = go.Scatter(x=np.arange(1, longest, 1/daemon_sample_rate), y=np.mean(cpu_resized, axis=0), name='avg-%s-cpu' % node_type, line=dict(width=1))
        fig.append_trace(cpu, 1, 1)
    
    # Show each minute of the attack as a colored box
    rand_colors = get_random_colors(12)
    start = 120
    shapes = []
    for i in range(11):
        shapes.append({
            'type': 'rect',
            'xref': 'x1',
            'yref': 'y1',
            'x0': start+60*i,
            'x1': start+60*(i+1),
            'y0': 0,
            'y1': 80,
            'line': {
                'width': 0,
            },
            'fillcolor': rand_colors[i],
            'opacity': 0.15
        })
    
    layout = go.Layout(
        height=1000,
        title='Daemon Data',
        shapes=shapes
    )
    fig['layout'].update(layout)
    iplot(fig, filename='daemon_data', link_text='Export')

In [9]:
daemon_stats = {}
files = [os.path.splitext(f) for f in sorted(os.listdir(daemon_path)) if os.path.isfile(daemon_path + f)]
for f in files:
    daemon_stats[f[0]] = get_daemon_stats_from_file(daemon_path + ''.join(f))
    
try:
    plot_daemon_stats_plotly(daemon_stats, [f[0] for f in files])
except PlotlyEmptyDataError:
    print('There was no daemon data at the specified location')

In [6]:
def plot_module_stats(stats):
    listen = go.Scatter(x=stats['Timestamp'], y=stats['listen_q'], name='Listen Queue')
    accept = go.Scatter(x=stats['Timestamp'], y=stats['accept_q'], name='Accept Queue')
    
    rand_colors = get_random_colors(12)
    start = 120
    shapes = []
    for i in range(11):
        shapes.append({
            'type': 'rect',
            'xref': 'xaxis',
            'yref': 'paper',
            'x0': start+60*i,
            'x1': start+60*(i+1),
            'y0': 0,
            'y1': 1,
            'line': {
                'width': 0,
            },
            'fillcolor': rand_colors[i],
            'opacity': 0.15
        })
        
    fig = go.Figure(data=[listen, accept], layout=go.Layout(
        height=400,
        autosize=True,
        title='Module Data',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Queue Entries'),
        showlegend=True,
        shapes=shapes
    ))
    iplot(fig, filename='module_data', link_text='Export')

In [7]:
module_stats = get_module_stats_from_file(module_path + ''.join('argus-module.txt'))
if module_stats:
    try:
        plot_module_stats(module_stats)
    except PlotlyEmptyDataError:
        print('There was no module data at the specified location')